# From the scanner to our computer

## Types of MR scans

### Anatomical

> Sourced from https://case.edu/med/neurology/NR/MRI%20Basics.htm

![Anatomical](imgs/anatomical.jpg)

* 3D image of anatomy (i.e., shape, volume, cortical thickness, brain region)
* can differentiate tissue types

### Functional

> Sourced from Wagner and Linguist, 2015

![BOLD](imgs/bold.gif)

![fmri](imgs/fmri_timeseries.png)

* tracks the blood oxygen level-dependent (BOLD) signal as an analogue of brain activity
* 4D image (x, y, z + time)

### Diffusion

> Sourced from http://brainsuite.org/processing/diffusion/tractography/

![diffusion](imgs/dwi.gif)

![tractography](imgs/dwi_tracts.png)

* measures diffusion of water in order to model tissue microstructure
* 3D image (x, y, z + direction of diffusion)
* need parameters about the strength of the diffusion "gradient" and its direction (`.bval` and `.bvec`)

## Neuroimaging File Formats


| Format Name | File Extension |                     Origin                     |
|:-----------:|:--------------:|:----------------------------------------------:|
|    DICOM    | none           |              ACR/NEMA Consortium               |
| Analyze     | .img/.hdr      | Analyze Software, Mayo Clinic                  |
| NIfTI       | .nii           | Neuroimaging Informatics Technology Initiative |
|    MINC     | .mnc           | Montreal Neurological Institute                |
| NRRD        | .nrrd          |                                                |

From the MRI scanner, images are initially collected in the DICOM format and can be converted to these other formats to make working with the data easier.

Let's download some example DICOM data to see what it looks like. This data was generously shared publicly by the [Princeton Handbook for Reproducible Neuroimaging](https://brainhack-princeton.github.io/handbook/).


In [1]:
from tqdm import tqdm
import requests
import os

if not os.path.isfile('data/0219191.tar.gz'):
    url = "https://zenodo.org/record/3677090/files/0219191_mystudy-0219-1114.tar.gz"
    response = requests.get(url, stream=True)

    with open("data/0219191.tar.gz", "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)
else:
    print("File exists. Go to the next step.")

File exists. Go to the next step.


Using `tarfile` package to unpack the data.

In [2]:
import tarfile
from glob import glob
file = tarfile.open("data/0219191.tar.gz", "r:gz")
if not os.path.exists('data/dicom_examples/'):
    os.makedirs('data/dicom_examples/')
file.extractall('./data/dicom_examples/')
file.close()


In [3]:
import gzip
import shutil
path = 'data/dicom_examples/0219191_mystudy-0219-1114/dcm/*.gz'

for i in glob(path):
    with gzip.open(i, 'rb') as f_in:
        with open('data/dicom_examples/0219191_mystudy-0219-1114/dcm/' + i.split('/')[-1].split('.')[0].split('\\')[-1] + 'dcm', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

NIfTI is one of the most ubiquitous file formats for storing neuroimaging data. We can convert our DICOM data to NIfTI using `dcm2niix`.

In [4]:
if not os.path.exists('data/dicom_examples/nii'):
    os.makedirs('data/dicom_examples/nii')

os.system('dcm2niix -z y -o data/dicom_examples/nii data/dicom_examples/0219191_mystudy-0219-1114/dcm/')

0